In [1]:
import dataclasses
from typing import *

In [2]:
def parse_input(fl):
    return open(fl).read().replace("\n", "").strip().split(",")

In [3]:
test = parse_input("data/day15-test.txt")
inputs = parse_input("data/day15-input.txt")
test

['rn=1',
 'cm-',
 'qp=3',
 'cm=2',
 'qp-',
 'pc=4',
 'ot=9',
 'ab=5',
 'pc-',
 'pc=6',
 'ot=7']

In [4]:
def part1(codes):
    return sum(sha256(c) for c in codes)

def sha256(code):
    tot = 0
    for ch in code:
        tot += ord(ch)
        tot *= 17
        tot %= 256
    return tot

In [5]:
part1(test), part1(inputs)

(1320, 506269)

In [6]:
@dataclasses.dataclass
class LN:
    val: Any = None
    prev: 'LN' = None
    next: 'LN' = None

    def __repr__(self):
        return f"LN{self.val}"

@dataclasses.dataclass
class HashMap:
    """Implement hashmap with lists of doubly linked-list
    """

    sz: int = 256
    # list of hashboxes containing head and tail of each node.
    _boxes: list[tuple[LN, LN]] = dataclasses.field(init=False)
    _box2lbl2node: list[dict[str, LN]] = dataclasses.field(init=False)

    def __post_init__(self):
        self._boxes = []
        for _ in range(self.sz):
            # sentinal is both head and tail
            s = LN()
            self._boxes.append((s, s))
        self._box2lbl2node = [{} for _ in range(self.sz)]

    def __setitem__(self, k, v):
        ix = sha256(k)
        lbl2node = self._box2lbl2node[ix]
        if k in lbl2node:
            lbl2node[k].val = (k, v)
        else:
            h, t = self._boxes[ix]
            new_node = LN(val=(k, v), prev=t)
            t.next = new_node
            self._boxes[ix] = (h, new_node)
            lbl2node[k] = new_node

    def __delitem__(self, k):
        ix = sha256(k)
        lbl2node = self._box2lbl2node[ix]
        if k not in lbl2node:
            return
        n = lbl2node.pop(k)
        # n.prev can't be none thanks to sentinel
        n.prev.next = n.next
        if n.next is not None:
            n.next.prev = n.prev
        else:
            # if n is the tail node
            h, t = self._boxes[ix]
            assert n==t
            self._boxes[ix] = (h, n.prev)
    
    def __repr__(self):
        return f"HM(\n{self._boxes=}\n\n{self._box2lbl2node=}\n)"

In [7]:
def part2(codes, hashmap_sz=256):
    hm = HashMap(sz=hashmap_sz)
    for c in codes:
        if '-' in c:
            del hm[c[:-1]]
        elif '=' in c:
            k, v = c.split('=')
            hm[k] = int(v)
        else:
            raise NotImplementedError()
    return {"hm": hm, "score": _score_hm(hm=hm)}

def _score_hm(hm):
    tot = 0
    for i, (bh, _) in enumerate(hm._boxes, start=1):
        slot_num = 1
        # skip sentinel
        c = bh.next
        while c is not None:
            # single lens
            tot += i * slot_num * c.val[1]
            slot_num += 1
            c = c.next
    return tot

In [8]:
p2_test_res = part2(codes=test, hashmap_sz=4)
p2_test_res

{'hm': HM(
 self._boxes=[(LNNone, LN('cm', 2)), (LNNone, LNNone), (LNNone, LNNone), (LNNone, LN('pc', 6))]
 
 self._box2lbl2node=[{'rn': LN('rn', 1), 'cm': LN('cm', 2)}, {}, {}, {'ot': LN('ot', 7), 'ab': LN('ab', 5), 'pc': LN('pc', 6)}]
 ),
 'score': 145}

In [9]:
p2_input_res = part2(codes=inputs)
p2_input_res['score']

264021